# Day 19: Not Enough Minerals

A classic resource and machine building strategy game.

First up, let's have a quick check of the size of the problem:

24 Minutes, with a minute-resolution simulation.
Puzzle input contains 30 blueprints.

The blueprints vary the quantity of the source materials required, but they don't change the type of resource.

## Approach

A first glance, this looks similar to the valves puzzle from a few days ago. I solved that using a brute-force technique, checking every single possible option (and discarding some obvious losers along the way to improve performance).
But there are so many more possibilities here, that taking that approach seems wrong. And also a bit repetative... I want to try something different.

So... how about having a strategy engine that works backwards from the goal?

Generic recipes:
a Ore              -> ore bot   -> Ore
b Ore              -> clay bot  -> Clay
c Ore + d Clay     -> obs bot   -> Obsidian
e Ore + f Obsidian -> geode bot -> Geode

Re-arrange to work backwards: (and ignoring time)
1 Geode = e Ore + f Obsidian
        = e Ore + f (c Ore + d Clay)
        = e Ore + fc Ore + fdb Ore 
        = ( e + fc + fdb ) Ore

Start with 1 ore collecting Robot.
Ore is the primary contraint... but we don't want an ore stock-pile at the end.


Other approaches to consider... could this be turned into a single function that can be optimised? 
What would the inputs be? The choice how many robots of each type to build in each minute.
But it's integer choices... so we're into integer programming - maybe try using: https://www.cvxpy.org/examples/applications/OOCO.html

But perhaps I should try a slightly more straightforward way first?

Many hours in, straightforward isn't working. 

Is this a clue in the puzzle?
"The elephants are starting to look hungry, so you shouldn't take too long; you need to figure out which blueprint would maximize the number of opened geodes after 24 minutes by figuring out which robots to build and when to build them."

Hungry suggests a greedy algorithm.
But already discovered that doens't work.

Urgghrr.... close to giving up. Do I just brute force it again?



In [2]:
testData ="""Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian."""

import re

def process(input:str):
    pattern = '(\d+)'
    for l in input.splitlines():
        match = re.findall(pattern,l)
        blueprint = [int(i) for i in match]
        print(blueprint)

#test
process(testData)

[1, 4, 2, 3, 14, 2, 7]
[2, 2, 3, 3, 8, 3, 12]


In [5]:
#optimised
ID = 0
ORE = 1
CLAY = 2
OBSIDIAN = 3
GEODE = 4

#friendly
# ID = 'ID'
# ORE = 'ore'
# CLAY = 'clay'
# OBSIDIAN = 'obsidian'
# GEODE = 'geode'

RUNTIME = 32

import copy 
from math import ceil
import random

class Puzzle:
    def blueprintsFrom(self, input):
        pattern = '(\d+)'
        blueprints = []
        for l in input.splitlines():
            match = re.findall(pattern,l)
            intarr = [int(i) for i in match]
            blueprint = {}
            blueprint[ID] = intarr[0]
            blueprint[ORE] = {ORE:intarr[1], CLAY:0, OBSIDIAN:0, GEODE:0}
            blueprint[CLAY] = {ORE:intarr[2], CLAY:0, OBSIDIAN:0, GEODE:0}
            blueprint[OBSIDIAN] = {ORE:intarr[3], CLAY:intarr[4], OBSIDIAN:0, GEODE:0}
            blueprint[GEODE] = {ORE:intarr[5], CLAY:0, OBSIDIAN:intarr[6], GEODE:0}
            blueprints.append(blueprint)
        return blueprints
    
    def __init__(self, input:str):
        self.blueprints = self.blueprintsFrom(input)
    
    def runPuzzle(self):
        total = 1
        for blueprintIndex in range(3):
            geodes = self.playGame(blueprintIndex)
            id = self.blueprints[blueprintIndex][ID]
            quality = id * geodes
            print('Completed game for blueprint ' + str(id))
            total = total * geodes
        print ('Total score for all blueprints: ' +str(total))

    def playGame(self, blueprintID:int)->int:
        #play all version of a game for a given blueprint and return max score
        rootgame = Game(self.blueprints[blueprintID])
        processQueue = []
        maxScore = 0
        botOptions = [GEODE,OBSIDIAN,CLAY,ORE]
        completedGames = 0
        prunedGames = 0
        for bot in botOptions:
            f = rootgame.fork(bot)
            processQueue.append(f) 

        while processQueue:
            totalgames = len(processQueue) + completedGames + prunedGames
            if totalgames % 10000000 == 0: #print update every thousand games
                print('Processing ' + str(len(processQueue))+ ' simulations, with pruning on: ' +str(maxScore) + ' Complete: ' + str(completedGames) + ' Pruned: '+str(prunedGames))
            game = processQueue.pop() #by taking off the bottom we have turned Queue into stack... this should give us depth first search, allowing us to cull worse answers quicker
            #fork it with every option of bot to build next
            #print('Running : ' + str(game))
            running = game.progressWithChoice()
            maxScore = max(maxScore,game.materials[GEODE])
            if running:
                if game.upperbound > maxScore:
                    #print('Qualified:' + str(game))
                    #the bot order of perference matters quite a lot... both extremes of order are clearly terrible. We might converge better by randomising
                    random.shuffle(botOptions)
                    for bot in botOptions:
                        f = game.fork(bot)
                        processQueue.append(f)
                else:
                    pass
                    #print('Pruned  : ' +str(game))    
                    prunedGames += 1
            else:
                pass
                #print('Complete: '+str(game))
                completedGames += 1
        return maxScore

class Game:
    def fork(self, nextBot:str):
        #fork a copy of the game
        f = Game(self.blueprint)
        f.time = self.time
        f.materials = copy.copy(self.materials)
        f.bots = copy.copy(self.bots)
        f.upperbound = self.upperbound
        f.nextBot = nextBot
        return f
    
    def __init__(self, blueprint):
        self.time = 0
        self.materials= {ORE:0, CLAY:0, OBSIDIAN:0, GEODE:0}
        self.bots= {ORE:1, CLAY:0, OBSIDIAN:0, GEODE:0}
        self.blueprint = blueprint
        #keep track of theoretical maximum GEODE count - will use this to prune branches
        self.upperbound = 0
        self.nextBot = 0

    def progressWithChoice(self)->bool:
        #pass in the choice of what bot to build next. Return True is there is still time on the clock, False is simulation complete
        #alway makes sense to build a bot next... because ultimate goal (geode splitting) doesn't require the consumpation of material, so it always makes sense to at least try to build another geoode bot
        botRecipe = self.blueprint[self.nextBot]
        timeRemaining = RUNTIME - self.time
        #calculate time required to build bot
        timeNeeded = 0
        for k, v in self.materials.items():
            if botRecipe[k] != 0:
                rateOfProduction = self.bots[k]
                if rateOfProduction==0:
                    #we're not building the bot
                    timeNeeded = RUNTIME + 1 #essentially forever
                else:
                    t = ceil((botRecipe[k] - v) / self.bots[k])
                timeNeeded = max(timeNeeded,t)
        
        #bot won't be ready until self.time + timeneeded + 1
        timeNeeded += 1    
        if timeNeeded > timeRemaining:
            #progress simulation to end of runtime
            for k,v in self.bots.items():
                self.materials[k] += v * timeRemaining

            self.time += timeRemaining
            return False
        else:
            #progress simualtion until bot is purchases
            for k,v in self.bots.items():
                self.materials[k] += v * timeNeeded
            #remove materials for bot build
            for k, v in self.materials.items():
                self.materials[k] -= botRecipe[k]
            #add bot
            self.bots[self.nextBot] += 1
            self.time += timeNeeded
            self.updateUpperBound()
            #print('T=' + str(self.time)+ ' Bots: ' + str(self.bots) + ' Materials: ' + str(self.materials) + ' Upperbound: ' + str(self.upperbound))
            return True
     
    def updateUpperBound(self):
        timeRemaining = RUNTIME - self.time
        #assume we build a geode bot per minute - we can come back and refine this is we want to be more brutal.
        self.upperbound = (timeRemaining + 1) * timeRemaining / 2 + self.materials[GEODE] + self.bots[GEODE] * timeRemaining

    def __str__(self):
        return 'T=' + str(self.time)+ ' Bots: ' + str(self.bots) + ' Materials: ' + str(self.materials) + ' Upperbound: ' + str(self.upperbound) + ' Next bot: ' + str(self.nextBot)
#tests
tp = Puzzle(testData)
#print(tp.blueprints)
print(tp.playGame(0))



Processing 32 simulations, with pruning on: 42 Complete: 159858 Pruned: 9840110
Processing 31 simulations, with pruning on: 42 Complete: 159858 Pruned: 9840111
Processing 30 simulations, with pruning on: 42 Complete: 159858 Pruned: 9840112
Processing 29 simulations, with pruning on: 42 Complete: 159858 Pruned: 9840113
Processing 28 simulations, with pruning on: 42 Complete: 159858 Pruned: 9840114
56


In [6]:
#puzzle input
puzzleinput = open('day19input.txt').read()
p = Puzzle(puzzleinput)
p.runPuzzle()


Processing 46 simulations, with pruning on: 13 Complete: 506145 Pruned: 9493809
Processing 45 simulations, with pruning on: 13 Complete: 506145 Pruned: 9493810
Processing 28 simulations, with pruning on: 16 Complete: 1655897 Pruned: 38344075
Processing 27 simulations, with pruning on: 16 Complete: 1655897 Pruned: 38344076
Processing 26 simulations, with pruning on: 16 Complete: 1655897 Pruned: 38344077
Processing 25 simulations, with pruning on: 16 Complete: 1655897 Pruned: 38344078
Processing 24 simulations, with pruning on: 16 Complete: 1655897 Pruned: 38344079
Processing 23 simulations, with pruning on: 16 Complete: 1655898 Pruned: 38344079
Completed game for blueprint 1
Processing 41 simulations, with pruning on: 31 Complete: 92124 Pruned: 9907835
Processing 40 simulations, with pruning on: 31 Complete: 92124 Pruned: 9907836
Processing 39 simulations, with pruning on: 31 Complete: 92124 Pruned: 9907837
Processing 38 simulations, with pruning on: 31 Complete: 92124 Pruned: 9907838
P